In [1]:
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertForSequenceClassification, BertConfig
import torch
import re
from collections import OrderedDict
import numpy as np
import json
import datetime

# 加载分类器 model1 用于语句分类  | model2 用于岗位匹配
# model_name = 'hfl/chinese-roberta-wwm-ext'
model_name = './bert-roberta'
tokenizer1 = BertTokenizer.from_pretrained(
    model_name)

# 使用时将模型放在文件夹下
config1 = BertConfig.from_pretrained(model_name)
config1.num_hidden_layers = 2
config1.num_labels = 7
model1 = BertForSequenceClassification.from_pretrained(
    model_name, config=config1)
model1.load_state_dict(torch.load('Roberta.pt'))
model1.eval()

tokenizer2 = BertTokenizer.from_pretrained(model_name)
config2 = BertConfig.from_pretrained(model_name)
config2.num_hidden_layers = 2
config2.num_labels = 11
model2 = BertForSequenceClassification.from_pretrained(
    model_name, config=config2)
model2.load_state_dict(torch.load('Job.pt'))
model2.eval()


# 加载NER
tokenizer3 = AutoTokenizer.from_pretrained(model_name)
pretrained = AutoModel.from_pretrained(model_name)


# 定义下游模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.tuning = False
        self.pretrained = None

        self.rnn = torch.nn.GRU(768, 768, batch_first=True)
        self.fc = torch.nn.Linear(768, 8)

    def forward(self, inputs):
        if self.tuning:
            out = self.pretrained(**inputs).last_hidden_state
        else:
            with torch.no_grad():
                out = pretrained(**inputs).last_hidden_state
        out, _ = self.rnn(out)
        out = self.fc(out).softmax(dim=2)
        return out


model3 = torch.load('Ner.pt')
model3.eval()
model3.to('cpu')


Some weights of the model checkpoint at ./bert-roberta were not used when initializing BertForSequenceClassification: ['bert.encoder.layer.9.attention.output.dense.weight', 'bert.encoder.layer.10.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.output.dense.weight', 'bert.encoder.layer.9.output.dense.bias', 'bert.encoder.layer.5.attention.self.query.bias', 'bert.encoder.layer.3.attention.self.value.weight', 'bert.encoder.layer.8.attention.output.LayerNorm.weight', 'bert.encoder.layer.5.intermediate.dense.bias', 'bert.encoder.layer.9.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.self.key.bias', 'bert.encoder.layer.11.attention.self.key.bias', 'bert.encoder.layer.8.attention.self.key.bias', 'bert.encoder.layer.5.output.LayerNorm.weight', 'bert.encoder.layer.4.intermediate.dense.weight', 'bert.encoder.layer.9.attention.self.key.bias', 'bert.encoder.layer.9.attention.output.LayerNorm.weight', 'bert.encoder.layer.5.attention.self.value.bias', 'bert.encoder.layer.6.atten

Model(
  (rnn): GRU(768, 768, batch_first=True)
  (fc): Linear(in_features=768, out_features=8, bias=True)
  (pretrained): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, ou

In [2]:
def label_predict(sentence):
    encoding = tokenizer1.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].squeeze()
    attention_mask = encoding['attention_mask'].squeeze()
    with torch.no_grad():
        outputs = model1(input_ids.unsqueeze(
            0), attention_mask=attention_mask.unsqueeze(0))
        logits = outputs.logits

    _, predicted_label = torch.max(logits, dim=1)

    return predicted_label.item()


def job_predict(sentence):
    encoding = tokenizer2.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].squeeze()
    attention_mask = encoding['attention_mask'].squeeze()
    with torch.no_grad():
        outputs = model2(input_ids.unsqueeze(
            0), attention_mask=attention_mask.unsqueeze(0))
        logits = outputs.logits

    # 获取预测的标签和对应的预测概率值
    predicted_probs = torch.softmax(logits, dim=1)
    # predicted_label = torch.argmax(predicted_probs, dim=1)

    predicted_labels = {}
    for index, value in enumerate(predicted_probs.squeeze().tolist()):
        # 设置概率阈值 超过该阈值的可以作为候选项 此处 0.1 较合理
        if value >= 0.1:
            predicted_labels[index] = value
    
    return dict(sorted(predicted_labels.items(), key=lambda x: x[1], reverse=True))


def ner_predict(sentence):
    if len(sentence) > 500:
        return [[] for _ in range(3)]
    inputs = tokenizer3.encode_plus([sentence],
                                    truncation=True,
                                    padding=True,
                                    return_tensors='pt',
                                    is_split_into_words=True)
    with torch.no_grad():
        outputs = model3(inputs)
    preds = outputs.argmax(dim=2)[0]
    result = ''
    res = [[] for _ in range(3)]
    tmp = ''
    current_flag = -1

    for i in range(len(preds)):
        if inputs['attention_mask'][0][i] == 1:
            result += tokenizer3.decode(inputs['input_ids'][0][i])+' '
            result += str(preds[i].item())+' '
            num = preds[i].item()
            # num 不为0和7和#表示该词为关键词
            if (num != 0 and num != 7 and num != '#'):
                # 关键词开始为奇数
                if (num & 1):
                    # 将形如 广5东6广5州6 拆成两个词
                    if (len(tmp) > 1):
                        res[(current_flag-1)//2].append(tmp)
                        tmp = ''
                    current_flag = num
                    tmp += tokenizer3.decode(inputs['input_ids'][0][i])

                    # 防止形如 X4X4 出现
                elif (num & 1 == 0 and current_flag != -1):
                    tmp += tokenizer3.decode(inputs['input_ids'][0][i])
            else:
                if (len(tmp) > 1):
                    # current_flag 1对应姓名下标0，3对应组织下表1，5对应地点下表2
                    res[(current_flag-1)//2].append(tmp)
                    tmp = ''
                    current_flag = -1
    return res


In [3]:
import info


def re_basedata(basic_data, data):
    if basic_data['tel'] == '':
        tel = re.findall(info.tel_pattern(), data)
        if tel:
            basic_data['tel'] = tel[0]
    if basic_data['email'] == '':
        email = re.findall(info.email_pattern(), data)
        if email:
            basic_data['email'] = email[0]
    if basic_data['age'] == 0:
        age = re.search(info.age_pattern(), data)
        if age:
            if age.group(1):
                basic_data['age'] = int(age.group(1))
            else:
                basic_data['age'] = int(age.group(2))
    if basic_data['birth'] == '':
        birth = re.search(info.birth_pattern(), data)
        if birth:
            basic_data['birth'] = birth.group()
            basic_data['age'] = 2023 - int(basic_data['birth'][:4])+1
    edu = re.findall(info.edu_pattern(), data)
    if edu:
        for e in edu:
            # 找到的学历大于当前的学历就更新
            if info.edu_map()[e] > info.edu_map()[basic_data['edu']]:
                basic_data['edu'] = e
    return basic_data


def handle_basedata(data, basic_data, total_data):
    # 合并语句 优化Ner次数
    for data in list(OrderedDict.fromkeys(data[0])):
        # 先跑NER 选出姓名 地点 学历
        output_prediction = ner_predict(data)
        if len(output_prediction[0]) and basic_data['name'] == '' and re.match(info.chinese_str(), output_prediction[0][0]):
            basic_data['name'] = output_prediction[0][0]
        elif len(output_prediction[1]):
            basic_data['college'].extend(output_prediction[1])
        elif len(output_prediction[2]):
            basic_data['loc'].extend(output_prediction[2])
        # 再跑正则匹配更新label
        basic_data = re_basedata(basic_data, data)

    fixed_college = []
    for x in basic_data['college']:
        for keyword in info.college_endword():
            keyword_position = x.find(keyword)
            if keyword_position != -1:
                fixed_college.append(x[:keyword_position + len(keyword)])
                # 注意这里内循环需要是 keyword 并且必须 break 防止识别两次 优先级见 info
                break
    basic_data['college'] = list(OrderedDict.fromkeys(fixed_college))
    basic_data['loc'] = list(OrderedDict.fromkeys(basic_data['loc']))[:2]

    # 给学校加tag
    if np.intersect1d(basic_data['college'], info.college985()).size > 0:
        total_data['tag']['edu_tag'].append('985')
        total_data['score'] += info.score_map()['985']
    elif np.intersect1d(basic_data['college'], info.college211()).size > 0:
        total_data['tag']['edu_tag'].append('211')
        total_data['score'] += info.score_map()['211']

    # 给籍贯加tag
    if len(basic_data['loc']) > 0:
        loc_pattern = '|'.join(info.province())
        for loc in basic_data['loc']:
            matches = re.findall(loc_pattern, loc)
            if matches:
                total_data['tag']['loc_tag'] = matches[0]
                break

    # 最高学历加tag
    if len(basic_data['edu']) > 0:
        total_data['tag']['edu_tag'].append(basic_data['edu'])
        total_data['score'] += info.score_map()[basic_data['edu']]

    # 删除空字段
    key_to_del = []
    for key, value in basic_data.items():
        if value == '' or value == [] or value == 0:
            key_to_del.append(key)
    for key in key_to_del:
        del basic_data[key]


def handle_job_obj(total_data):
    raw_jobs = total_data['job_obj']
    fixed_jobs = []
    # 找首个个特殊符号 直接截断后半段加入答案 没找到符号直接加入答案
    for job in raw_jobs:
        pos = job.find(":")
        if pos == -1:
            pos = job.find("/")
        if pos == -1:
            pos = job.find("：")
        if pos == -1:
            fixed_jobs.append(job.replace(" ", ""))
        else:
            fixed_jobs.append(job[pos+1:].replace(" ", ""))
    fixed_jobs = list(OrderedDict.fromkeys(fixed_jobs))
    total_data['job_obj'] = fixed_jobs


def calculate_date_interval(date1, date2):
    date1_obj = datetime.datetime.strptime(date1, "%Y.%m")
    date2_obj = datetime.datetime.strptime(date2, "%Y.%m")
    diff = date2_obj - date1_obj
    years = diff.days // 365
    months = (diff.days % 365) // 30
    return years, months


def re_date(date):
    date = date.replace("年", ".").replace("月", "")
    if date[-1:] == '年':
        date = date[:-1]
    if '.' not in date:
        date = date + '.01'
    return date


def handle_experience(total_data):
    company = []
    year_interval = 0
    month_interval = 0
    for exp in total_data['experience']:
        # 跑NER找出所有工作单位
        output_prediction = ner_predict(exp)
        company.extend(output_prediction[1])
        # 计算工作年限
        matches = re.findall(info.work_time(), exp)
        if matches:
            for match in matches:
                date1 = re_date(match[0])
                date2 = '2023.04' if match[1] == '今' or match[1] == '至今' else re_date(
                    match[1])
                try:
                    years, months = calculate_date_interval(date1, date2)
                    if years >= 0:
                        year_interval += years
                        month_interval += months
                except Exception as e:
                    print(e)

        year_interval += month_interval // 12
        month_interval %= 12

    # 受限于Ner的准确性 仅在自定义下的关键字下识别
    # FIXME 择优选择下面两种之一
    # fixed_company = []
    # for x in company:
    #     for keyword in info.college_endword():
    #         keyword_position = x.find(keyword)
    #         if keyword_position != -1 and x[:keyword_position+len(keyword)] != keyword:
    #             fixed_company.append(x[:keyword_position+len(keyword)])
    fixed_company = [x for keyword in info.company_endword()
                     for x in company if x.endswith(keyword) and x != keyword]
    fixed_company = list(OrderedDict.fromkeys(fixed_company))
    total_data['tag']['experience_tag'].extend(fixed_company)
    if year_interval < 60 and year_interval >= 0:
        total_data['tag']['total_work_time'] = year_interval if month_interval == 0 else year_interval + 1
    else:
        total_data['tag']['total_work_time'] = 0
    # 工作经验分值计算 工作单位和工作年限权重为 1.5
    total_data['score'] += 1.2 * \
        len(fixed_company) + 0.1 * (year_interval * 12 + month_interval)


def handle_ability(total_data):
    # 提前判断是否已经找到了 优化一下性能
    CET6_flag, CET4_flag, Photoshop_flag, Office_flag, NCRE_flag = False, False, False, False, False
    CET6_patter = '|'.join(info.self_ability()[0])
    CET4_patter = '|'.join(info.self_ability()[1])
    Photoshop_patter = '|'.join(info.self_ability()[2])
    Office_patter = '|'.join(info.self_ability()[3])
    NCRE_patter = '|'.join(info.self_ability()[4])
    # 找出CET Photoshop Office 计算机等级考试 的tag
    for item in total_data['ability']:
        if CET6_flag == False:
            matches = re.findall(CET6_patter, item)
            if matches:
                total_data['tag']['ability'].append('CET6')
                total_data['score'] += info.score_map()['CET6']
                CET6_flag = True
        if CET6_flag == False and CET4_flag == False:
            matches = re.findall(CET4_patter, item)
            if matches:
                total_data['tag']['ability'].append('CET4')
                total_data['score'] += info.score_map()['CET4']
                CET4_flag = True
        if Photoshop_flag == False:
            matches = re.findall(Photoshop_patter, item)
            if matches:
                total_data['tag']['ability'].append('Photoshop')
                Photoshop_flag = True
        if Office_flag == False:
            matches = re.findall(Office_patter, item)
            if matches:
                total_data['tag']['ability'].append('Office办公软件')
                Office_flag = True
        if NCRE_flag == False:
            matches = re.findall(NCRE_patter, item)
            if matches:
                total_data['tag']['ability'].append('计算机等级考试')
                total_data['score'] += info.score_map()['NCRE']
                NCRE_flag = True


def handle_job_fit(total_data):
    work_time = total_data['tag']['total_work_time']
    edu = total_data['basic_data']['edu'] if 'edu' in total_data['basic_data'] else ''
    experience = ''.join(total_data['experience'])
    office_ability = True if re.findall(
        '|'.join(info.self_ability()[3]), ''.join(total_data['ability'])) else False
    age = total_data['basic_data']['age'] if 'age' in total_data['basic_data'] else 0

    predicted_labels = job_predict(experience)
    for key in predicted_labels:
        # 概率比 '暂无' 更小的就没有匹配的必要
        if key == 0:
            break
        if info.edu_map()[edu] >= info.edu_map()[info.job_fit()[key][1]] and work_time >= info.job_fit()[key][2] and (office_ability or not office_ability and info.job_fit()[key][3]) and age >= info.job_fit()[key][4]:
            total_data['job_fit'].append(info.job_fit()[key][0])
    if len(total_data['job_fit']) == 0:
        total_data['job_fit'].append('暂无')


def analysis(sentences):
    sentences = list(OrderedDict.fromkeys(sentences))
    # sequence 为最初分类序列 经过上下文纠正算法得到最终分类结果 data
    sequence = []
    data = [[] for _ in range(7)]
    for s in sentences:
        p = label_predict(s)
        s = re.sub(r'[\s\t]{2,}', ' ', s.strip())
        sequence.append([s, p])

    # 上下文纠正算法 纠正特殊标签6 这里有两种可能 6,2->6,1 或 6,2->2,2
    for i in range(0, len(sequence)-1):
        if sequence[i][1] == 6 and sequence[i+1][1] == 2:
            job_obj_patter = '|'.join(info.job_obj_keywords())
            matches = re.findall(job_obj_patter, sequence[i][0])
            if matches:
                # 6,2->6,1
                sequence[i+1][1] = 1
            else:
                # 6,2->2,2
                sequence[i][1] = 2

    # 至此分类完成
    for s in sequence:
        data[s[1]].append(s[0])

    # 对基本信息单独NER+正则匹配
    basic_data = {
        'name': '',
        'birth': '',
        'age': 0,
        'tel': '',
        'email': '',
        'college': [],
        'loc': [],
        'edu': ''
    }

    tag = {
        'edu_tag': [],
        'loc_tag': '',
        'experience_tag': [],
        'ability': [],
        'total_work_time': 0
    }

    total_data = {
        'basic_data': basic_data,
        'job_obj': data[1],
        'experience': data[2],
        'award': list(OrderedDict.fromkeys(data[3])),
        'ability': list(OrderedDict.fromkeys(data[4])),
        'job_fit': [],
        'tag': tag,
        'score': 0,
        'custom_content': {
            'money_obj': '',
            'self_desc': [],
            'self_tag': []
        }
    }

    handle_basedata(data, basic_data, total_data)
    handle_job_obj(total_data)
    handle_experience(total_data)
    handle_ability(total_data)
    handle_job_fit(total_data)

    # return json.dumps(total_data, ensure_ascii=False)
    return total_data


In [ ]:
import os
import docx2txt
import tqdm

res = {}
for i in tqdm.notebook.tqdm(range(101, 301)):
    path = f'D:\暂存\测试数据及说明\data\{i}.docx'
    if not os.path.isfile(path):
        continue

    # 读取Word文档
    text = docx2txt.process(path)
    lines = text.splitlines()
    stripped_lines = [line.strip() for line in lines]
    new_list = [x for x in stripped_lines if x]
    data = analysis(new_list)
    ans = {}
    ans['name'] = data['basic_data']['name'] if 'name' in data['basic_data'] else ''
    ans['age'] = data['basic_data']['age'] if 'age' in data['basic_data'] else ''
    ans['education'] = data['basic_data']['edu'] if 'edu' in data['basic_data'] else ''
    ans['school'] = data['basic_data']['college'][0] if 'college' in data['basic_data'] and len(
        data['basic_data']['college']) > 0 else ''
    ans['work_time'] = data['tag']['total_work_time']
    ans['match_position'] = data['job_fit']
    res[i] = ans
    print(ans)

json_data = json.dumps(res, ensure_ascii=False)
with open('./res.json', "w", encoding='utf-8') as file:
    file.write(json_data)


In [4]:
sentences1 = ['张吉惟','EXCEL','PPT','WORD','AI','PS','个人技能','出生年月：1998.11','性 别：女','籍 贯：北京海淀区','学 历：本科','政治面貌：党员','身 高：178cm','体 重：50kg','工作经历','自我评价','个人爱好','本人性格开朗、稳重、有活力，待人热情、真诚。有较强的组织能力、团体协作精神，较好的社交能力，善于处理各种人际关系。能迅速的适应各种环境，并融合其中。能把企业当作家庭，企业的财富就是我的财富，在努力为企业服务的过程中实现自身价值。','zhangjiwei@163.com','138 9999 9999','求职意向：行政管理类','基本信息','2016\t全国大学生英语竞赛一等奖','2017\t国家级一等奖学金','2018\t大学英语6级证书（CET-6）','奖励证书','教育背景','2015-2019\t国际经济与贸易','北京师范大学','政治经济学、西方经济学、国际经济学、计量经济学、世界经济概论、国际贸易理论与实务、国际金融、国际结算。','2022.3至今\t部长助理','北京猫眼机械有限公司','质管部部长助理.负责票证车间生产质量检查,控制,兼任企业内审员,推行ISO9001:2008及ISO14000体系并年审。','2021.2-2022.3\t总经理助理','北京猫眼国际货运代理有限公司','根据公司的经营理念和发展战略制定公司各工作岗位的工作目标;负责制定及推进公司员工的培训、绩效、薪酬的管理。','2020.2-2021.2\t运营助理','北京猫眼文化传媒有限公司','根据目标管理综合考评内容，深入、细致地到所跟踪的各场所了解、掌握情况，帮助和促进各场所提高行政、后勤管理水平。']
sentences2 = ['王美珠', '求职目标：市场总监-专注品牌方向', '王美珠', '求职目标：市场总监-专注品牌方向', '出生：1989.05', '住址：湖北省武汉市', '电话：13800138000', '邮箱：service@500d.me', '出生：1989.05', '住址：湖北省武汉市', '电话：13800138000', '邮箱：service@500d.me', '教育背景', '2014.07-2019.06                        湖南师范大学                  市场营销（博士）', '主修课程：管理学、微观经济学、宏观经济学、管理信息系统、统计学、会计学、财务管理、市场营销、经济法、消费者行为学、国际市场营销', '20011.07-2014.06                        湖南师范大学                  市场营销（硕士）', '主修课程：管理学、微观经济学、宏观经济学、管理信息系统、统计学、会计学、财务管理、市场营销、经济法、消费者行为学、国际市场营销', '教育背景', '2014.07-2019.06                        湖南师范大学                  市场营销（博士）', '主修课程：管理学、微观经济学、宏观经济学、管理信息系统、统计学、会计学、财务管理、市场营销、经济法、消费者行为学、国际市场营销', '20011.07-2014.06                        湖南师范大学                  市场营销（硕士）', '主修课程：管理学、微观经济学、宏观经济学、管理信息系统、统计学、会计学、财务管理、市场营销、经济法、消费者行为学、国际市场营销', '工作经历', '2022.09-至今                           辉世设计控股集团                  副总监', '负责协助集团旗下事业部开展各项工作，制定品牌传播方案；', '结合集团与事业部发展，制定营销策略、广告策略、品牌策略和公关策略，并组织推进执行；', '制定和执行媒体投放计划，跟踪和监督媒体投放效果，进行数据分析与撰写报告；', '负责公司品牌形象和价值提升的持续优化，提高品牌知名度；', '研究行业发展动态，定期进行市场调查,为产品更新提供建议。', '工作经历', '2022.09-至今                           辉世设计控股集团                  副总监', '负责协助集团旗下事业部开展各项工作，制定品牌传播方案；', '结合集团与事业部发展，制定营销策略、广告策略、品牌策略和公关策略，并组织推进执行；', '制定和执行媒体投放计划，跟踪和监督媒体投放效果，进行数据分析与撰写报告；', '负责公司品牌形象和价值提升的持续优化，提高品牌知名度；', '研究行业发展动态，定期进行市场调查,为产品更新提供建议。', '校园经历', '2011.09-2012.06                        湖南师范大学                  辩论队（队长）', '任职描述：', '负责50余人团队的日常训练、选拔及团队建设；', '作为负责人对接多项商业校园行活动，如《奔跑吧兄弟》大学站录制、长安福特全球校园行、《时代周末》校园行；', '代表湖南师范大学辩论队参与多项国际型比赛，并取得多项荣誉：', '湖南师范大学“唇舌烽火”辩论赛冠军、亚军；', '湖南师范大学2011级“杰出辩手”；', '2012.11-2014.06                        沟通与交流协会                  创始人/副会长', '任职描述：', '协助湖南省沟通协会湖南师范大学分部，从零开始组建初期团队；', '策划协会会员制，选拔、培训协会导师，推出一系列沟通课程；配合其他社团组织，承办诸如演讲、主持、讲坛等省级/校级活动；', '校园经历', '2011.09-2012.06                        湖南师范大学                  辩论队（队长）', '任职描述：', '负责50余人团队的日常训练、选拔及团队建设；', '作为负责人对接多项商业校园行活动，如《奔跑吧兄弟》大学站录制、长安福特全球校园行、《时代周末》校园行；', '代表湖南师范大学辩论队参与多项国际型比赛，并取得多项荣誉：', '湖南师范大学“唇舌烽火”辩论赛冠军、亚军；', '湖南师范大学2011级“杰出辩手”；', '2012.11-2014.06                        沟通与交流协会                  创始人/副会长', '任职描述：', '协助湖南省沟通协会湖南师范大学分部，从零开始组建初期团队；', '策划协会会员制，选拔、培训协会导师，推出一系列沟通课程；配合其他社团组织，承办诸如演讲、主持、讲坛等省级/校级活动；', '自我评价', '拥有多年的市场管理及品牌营销经验，卓越的规划、组织、策划、方案执行和团队领导能力，积累较强的人际关系处理能力和商务谈判技巧，善于沟通，具备良好的合作关系掌控能力与市场开拓能力；敏感的商业和市场意识，具备优秀的资源整合能力、业务推进能力； 思维敏捷，有培训演讲能力，懂激励艺术，能带动团队的积极性；擅长协调平衡团队成员的竞争与合作的关系，善于通过培训提高团队综合能力和凝聚力。',
              '自我评价', '拥有多年的市场管理及品牌营销经验，卓越的规划、组织、策划、方案执行和团队领导能力，积累较强的人际关系处理能力和商务谈判技巧，善于沟通，具备良好的合作关系掌控能力与市场开拓能力；敏感的商业和市场意识，具备优秀的资源整合能力、业务推进能力； 思维敏捷，有培训演讲能力，懂激励艺术，能带动团队的积极性；擅长协调平衡团队成员的竞争与合作的关系，善于通过培训提高团队综合能力和凝聚力。', '掌握技能', '普通话一级甲等', '通过全国计算机二级考试，熟练运用office相关软件。', '熟练使用绘声绘色软件，剪辑过各种类型的电影及班级视频。', '大学英语四/六级（CET-4/6），良好听说读写能力，快速浏览英语专业书籍。', '掌握技能', '普通话一级甲等', '通过全国计算机二级考试，熟练运用office相关软件。', '熟练使用绘声绘色软件，剪辑过各种类型的电影及班级视频。', '大学英语四/六级（CET-4/6），良好听说读写能力，快速浏览英语专业书籍。', '项目经历', '信健设计集团品牌升级发布会', '集团全新品牌logo及VI上线，在多渠道进行了传播；', '企业VIP客户群体逾60人，结合了线上发布、线下体验；', '后续媒体报道持续升温，子品牌罄玉结合代言人罗嘉良制造话题营销，为期3周；', '鼎基设计商业模式发布会', '整场活动以会议+洽谈双重模式进行，首日以介绍鼎基内部平台资源优势，政府背景优势等为主，一对多推介会进行推广普及；', '现场签署地方合作意向书，如：新疆、江西、浙江等优秀企业商户；', '以中国的波尔多为宣传点，主推旗下新疆大型项目，罄玉生态葡萄庄园、沙漠主题俱乐部等，制造营销、品牌热点。', '项目经历', '信健设计集团品牌升级发布会', '集团全新品牌logo及VI上线，在多渠道进行了传播；', '企业VIP客户群体逾60人，结合了线上发布、线下体验；', '后续媒体报道持续升温，子品牌罄玉结合代言人罗嘉良制造话题营销，为期3周；', '鼎基设计商业模式发布会', '整场活动以会议+洽谈双重模式进行，首日以介绍鼎基内部平台资源优势，政府背景优势等为主，一对多推介会进行推广普及；', '现场签署地方合作意向书，如：新疆、江西、浙江等优秀企业商户；', '以中国的波尔多为宣传点，主推旗下新疆大型项目，罄玉生态葡萄庄园、沙漠主题俱乐部等，制造营销、品牌热点。', '2021.09-2022.09                        鼎基设计有限公司                 市场及运营总监', '根据公司发展情况进行战略调整，配合前端销售部门搭建销售渠道；', '负责公司品牌形象和价值提升的持续优化，提高品牌知名度；', '研究行业发展动态，定期进行市场调查,为产品更新提供建议；', '负责公司部门(营运、品牌策划)制度规范，负责组织及监管市场部关于对外合作、渠道管理、媒体合作、推广策划以相关工作的落实。', '2020.09-2021.09                        宏建设计俱乐部                   市场副总监', '负责事业部产品对外推广和宣传，制定各种整合营销的活动；', '执行媒体投放计划，跟踪和监督媒体投放效果，进行数据分析撰写报告；', '向市场总监提供营销支持，并协助相关的公关事宜。', '2019.09-2020.09                        信健设计集团                     市场部-市场经理', '负责集团SEO优化事宜，在未使用百度竞价排名服务前提下，连接占据首页4至12条；', '参与策划集团年度、季度、月度线下推广活动方案；', '负责线下大型分享会”思享潭“的执行与后期线上推广。', '2021.09-2022.09                        鼎基设计有限公司                 市场及运营总监', '根据公司发展情况进行战略调整，配合前端销售部门搭建销售渠道；', '负责公司品牌形象和价值提升的持续优化，提高品牌知名度；', '研究行业发展动态，定期进行市场调查,为产品更新提供建议；', '负责公司部门(营运、品牌策划)制度规范，负责组织及监管市场部关于对外合作、渠道管理、媒体合作、推广策划以相关工作的落实。', '2020.09-2021.09                        宏建设计俱乐部                   市场副总监', '负责事业部产品对外推广和宣传，制定各种整合营销的活动；', '执行媒体投放计划，跟踪和监督媒体投放效果，进行数据分析撰写报告；', '向市场总监提供营销支持，并协助相关的公关事宜。', '2019.09-2020.09                        信健设计集团                     市场部-市场经理', '负责集团SEO优化事宜，在未使用百度竞价排名服务前提下，连接占据首页4至12条；', '参与策划集团年度、季度、月度线下推广活动方案；', '负责线下大型分享会”思享潭“的执行与后期线上推广。']
sentences3 = ['蔡依婷', '蔡依婷', '姓名：蔡依婷', '30岁', '籍贯：上海', '民族：汉族', '学历：本科', '专业：设计', '姓名：蔡依婷', '年龄：30岁', '籍贯：上海', '民族：汉族', '学历：本科', '专业：设计', '求 职 意 向 ： 设 计 师', '求 职 意 向 ： 设 计 师', '联系方式', '联系方式', '获得证书', '获得证书', '2011 - 2015', '本科', '2011 - 2015', '本科', '工作经历', '工作经历', 'Adobe认证设计师', 'Adobe认证产品专家', '平面设计师资格证书', '平面设计师资格证书', '英语四六级证书', 'Adobe认证设计师', 'Adobe认证产品专家', '平面设计师资格证书', '平面设计师资格证书', '英语四六级证书', '校园活动', '校园活动', '教育背景', '教育背景', '北京理工大学', '设计专业', '北京理工大学', '设计专业', '13912345678', 'xianxian@taobao.com', '13912345678', 'xianxian@taobao.com', '大学生三下乡', '在湖南省湘西自治州保靖县木芽村小学义务支教活动，被评为优秀支教老师。修改为自己活动内容介绍。修改为自己内容介绍。', '大学生三下乡', '在湖南省湘西自治州保靖县木芽村小学义务支教活动，被评为优秀支教老师。修改为自己活动内容介绍。修改为自己内容介绍。', '党支部书记', '修改成简单的一两句话介绍相关活动经历，活动的内容和获得的成绩。修改为自己活动内容介绍。', '党支部书记', '修改成简单的一两句话介绍相关活动经历，活动的内容和获得的成绩。修改为自己活动内容介绍。', '英语俱乐部干事', '修改成简单的一两句话介绍相关活动经历，活动的内容和获得的成绩。修改为自己活动内容介绍。', '英语俱乐部干事', '修改成简单的一两句话介绍相关活动经历，活动的内容和获得的成绩。修改为自己活动内容介绍。', '学生会主席', '修改成简单的一两句话介绍相关活动经历，活动的内容和获得的成绩。修改为自己活动内容介绍。修改为自己活动内容介绍。', '学生会主席', '修改成简单的一两句话介绍相关活动经历，活动的内容和获得的成绩。修改为自己活动内容介绍。修改为自己活动内容介绍。', '2015.9 – 2019.9', '上海', '2015.9 – 2019.9', '上海', '2019.9 – 至今', '上海', '2019.9 – 至今', '上海', '智扬顾问有限公司', '平面设计师', '智扬顾问有限公司', '平面设计师', '负责企业画册设计、标志设计、产品包装设计、杂志广告跨页设计、户外广告', '对公司的网站后台周期性的修改和维护', '对公司所有的宣传广告全程负责一直到完成阶段', '负责企业画册设计、标志设计、产品包装设计、杂志广告跨页设计、户外广告', '对公司的网站后台周期性的修改和维护', '对公司所有的宣传广告全程负责一直到完成阶段', '美铭顾问有限公司', '平面设计师', '美铭顾问有限公司', '平面设计师', '负责企业画册设计、标志设计、产品包装设计、杂志广告跨页设计、户外广告', '对公司的网站后台周期性的修改和维护', '对公司所有的宣传广告全程负责一直到完成阶段',
              '负责企业画册设计、标志设计、产品包装设计、杂志广告跨页设计、户外广告', '对公司的网站后台周期性的修改和维护', '对公司所有的宣传广告全程负责一直到完成阶段', '求 职 意 向 ： 设 计 师', '求 职 意 向 ： 设 计 师', '蔡依婷', '蔡依婷', '13912345678', 'xianxian@taobao.com', '13912345678', 'xianxian@taobao.com', '姓名：奈森设计', '年龄：24岁', '籍贯：上海', '民族：汉族', '学历：本科', '专业：设计', '姓名：奈森设计', '年龄：24岁', '籍贯：上海', '民族：汉族', '学历：本科', '专业：设计', '联系方式', '联系方式', '个人爱好', '个人爱好', '自我评估', '自我评估', '爬山', '写作', '音乐', '瑜伽', '爬山', '写作', '音乐', '瑜伽', '独立负责过大型项目，非常成功的规划了项目 A 工作；', 'B 项目，从开始到完成，极大地加强了我的逻辑思考能力；', '具备艺术家的灵感创作能力，公司公认的首席 PPT 制作人；', '非常优秀的用户心理洞察力，配合丰富的创意及文案能力极大地提高了我的执行力。段前间距设为0.2行。', '专业知识的学习以及多年的工作实践，使我积累了丰富的工作经验，并取得了优秀的销售业绩。段前间距设为0.2行。', '独立负责过大型项目，非常成功的规划了项目 A 工作；', 'B 项目，从开始到完成，极大地加强了我的逻辑思考能力；', '具备艺术家的灵感创作能力，公司公认的首席 PPT 制作人；', '非常优秀的用户心理洞察力，配合丰富的创意及文案能力极大地提高了我的执行力。段前间距设为0.2行。', '专业知识的学习以及多年的工作实践，使我积累了丰富的工作经验，并取得了优秀的销售业绩。段前间距设为0.2行。', 'MS Office', 'Photoshop', 'Illustrator', '英语', '日语', '粤语', 'MS Office', 'Photoshop', 'Illustrator', '英语', '日语', '粤语', '2011年国家励志一等奖学金', '2012年北京理工大学挑战主持人大赛第一名', '2012年北京理工大学演讲比赛三等奖', '2013年全国大学生数学建模大赛国家二等奖', '2013年担任英语俱乐部主席期间，领导英语俱乐部获得“某年度十佳社团”荣誉称号', '2014年北京理工大学第九届数学建模大赛一等奖', '2015年参加省红十字协会与校团委主办的演讲比赛并取得前三名的佳绩（全校近200人参赛）', '2011年国家励志一等奖学金', '2012年北京理工大学挑战主持人大赛第一名', '2012年北京理工大学演讲比赛三等奖', '2013年全国大学生数学建模大赛国家二等奖', '2013年担任英语俱乐部主席期间，领导英语俱乐部获得“某年度十佳社团”荣誉称号', '2014年北京理工大学第九届数学建模大赛一等奖', '2015年参加省红十字协会与校团委主办的演讲比赛并取得前三名的佳绩（全校近200人参赛）', '精通', '熟练', '熟练', '流利', '一般', '流利', '精通', '熟练', '熟练', '流利', '一般', '流利', '掌握技能', '掌握技能', '获奖经历', '获奖经历']
print(analysis(sentences1))
print(analysis(sentences2))
print(analysis(sentences3))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'basic_data': {'name': '张吉惟', 'birth': '1998.11', 'age': 26, 'tel': '138 9999 9999', 'email': 'zhangjiwei@163.com', 'college': ['北京师范大学'], 'loc': ['北京', '海淀区'], 'edu': '本科'}, 'job_obj': ['行政管理类'], 'experience': ['2022.3至今\t部长助理', '北京猫眼机械有限公司', '质管部部长助理.负责票证车间生产质量检查,控制,兼任企业内审员,推行ISO9001:2008及ISO14000体系并年审。', '2021.2-2022.3\t总经理助理', '北京猫眼国际货运代理有限公司', '根据公司的经营理念和发展战略制定公司各工作岗位的工作目标;负责制定及推进公司员工的培训、绩效、薪酬的管理。', '2020.2-2021.2\t运营助理', '北京猫眼文化传媒有限公司', '根据目标管理综合考评内容，深入、细致地到所跟踪的各场所了解、掌握情况，帮助和促进各场所提高行政、后勤管理水平。'], 'award': ['2016\t全国大学生英语竞赛一等奖', '2017\t国家级一等奖学金'], 'ability': ['EXCEL', 'PPT', 'WORD', 'AI', 'PS', '2018\t大学英语6级证书（CET-6）'], 'job_fit': ['人力资源管理', '文员'], 'tag': {'edu_tag': ['985', '本科'], 'loc_tag': '北京', 'experience_tag': ['北京猫眼机械有限公司', '北京猫眼国际货运代理有限公司', '北京猫眼文化传媒有限公司'], 'ability': ['Office办公软件', 'Photoshop', 'CET6'], 'total_work_time': 4}, 'score': 27.3, 'custom_content': {'money_obj': '', 'self_desc': [], 'self_tag': []}}
{'basic_data': {'name': '王美珠', 'birth': '1989.05', 'age': 35, '

In [ ]:
import docx2txt
import info
import tqdm

for i in tqdm.notebook.tqdm(range(1, 101)):
    if i == 65:
        continue
    text = docx2txt.process(f"D:\\暂存\\CV\\{i}.docx")
    lines = text.splitlines()
    stripped_lines = [line.strip('\t').replace('\t', ' ')
                        for line in lines]
    sentences = [x for x in stripped_lines if x.strip() != '']
    sequence = []
    data = [[] for _ in range(7)]
    for s in sentences:
        p = label_predict(s)
        s = re.sub(r'[\s\t]{2,}', ' ', s.strip())
        sequence.append([s, p])

    # 上下文纠正算法 纠正特殊标签6 这里有两种可能 6,2->6,1 或 6,2->2,2
    for j in range(0, len(sequence)-1):
        if sequence[j][1] == 6 and sequence[j+1][1] == 2:
            job_obj_patter = '|'.join(info.job_obj_keywords())
            matches = re.findall(job_obj_patter, sequence[j][0])
            if matches:
                # 6,2->6,1
                sequence[j+1][1] = 1
            else:
                # 6,2->2,2
                sequence[j][1] = 2

    # 至此分类完成
    for s in sequence:
        data[s[1]].append(s[0])

    # job = ''.join(data[2])+'&'+str(mp[i])+'\n'
    # file.write(job)
    print(analysis(''.join(data[2])))

    # print(f"简历{i} - {analysis(new_list)['job_fit']}")


In [ ]:
# ner测试
input_sentence = '北京时光印象信息科技有限公司 推广组实习生 2016.03-2016.5'
output_prediction = ner_predict(input_sentence)
print(f'姓名{list(output_prediction[0])},组织{list(output_prediction[1])},地点{list(output_prediction[2])}')


In [ ]:
# 分类测试
label = {
    0: '基本信息',
    1: '求职意向',
    2: '工作/项目经历',
    3: '获得奖项',
    4: '个人能力',
    5: '垃圾信息',
    6: '重要标注'
}
sentences = [
    '重庆大学-市场营销',
    '恩汇信息科技有限公司 产品经理',
    'Reaaasdqweeeeeefsdsdfasdasd', 'adsdasfvcasDDa', '中国石油大学  物理系',
    '2018.05-2019.06   辩论队（队长）',
    '许爱礼',
    '根据公司发展情况进行战略调整，配合前端销售部门搭建销售渠道；',
    '东城区公安局某某分局政工监督室                                 2017.08-2018.08',
    '风雨工作室                                               2018.08-2021.02',
'2018.2-2018.12做过一家小型民营企业的兼职行政文员，对行政文资工作有了初步的认识。实习经历也写在此栏目。',
'大二期间担任系学生会生活部部长，锻炼了自己的组织协调能力；实习经历也写在此栏目。段前间距设为0.3行',
'大一、大二期间担任校市场研究会干事，参与组织多次研讨会；实习经历也写在此栏目。段前间距设为0.3行。',
'大三期间担任学生会生活部部长，组织多次校园活动，都取得了不错的效果；实习经历也写在此栏目',
'利用大二署假参加学校组织的“三下乡”活动，加深了对社会的了解。段前间距设为0.3行。',

]

for s in sentences:
    p = label_predict(s)
    print(f'{s} -- {label[p]}')
